In [1]:
from kindra import kindle, constants
import polars as pl 
from polars import col 
import os
# from dotenv import load_dotenv

# load_dotenv()

In [ ]:
# what about joining on timestamp? could join words and lookup that way. 
# no, this doesn't work. 

In [17]:
info = kindle.query_db("select * from BOOK_INFO")
words = kindle.query_db("select * from WORDS")
lookup = kindle.query_db("select * from LOOKUPS").drop("id")

In [18]:
info.columns

['id', 'asin', 'guid', 'lang', 'title', 'authors']

In [19]:
words.columns

['id', 'word', 'stem', 'lang', 'category', 'timestamp', 'profileid']

In [20]:
lookup.columns

['word_key', 'book_key', 'dict_key', 'pos', 'usage', 'timestamp']

In [23]:
words.head()

id,word,stem,lang,category,timestamp,profileid
str,str,str,str,i64,i64,str
"""fr:règles""","""règles""","""règle""","""fr""",0,1677623350236,""""""
"""fr:privées""","""privées""","""privé""","""fr""",0,1677623396944,""""""
"""fr:fouet""","""fouet""","""fouet""","""fr""",0,1677623430942,""""""
"""fr:éclairage""","""éclairage""","""éclairage""","""fr""",0,1677623467371,""""""
"""fr:rébarbative""","""rébarbative""","""rébarbatif""","""fr""",0,1677623634277,""""""


In [28]:
bla = info.select(["id", "title"]
    ).join(lookup, left_on="id", right_on="book_key"
    ).join(words.drop("timestamp"), left_on="word_key", right_on="id"
    )

In [29]:
bla.columns

['id',
 'title',
 'word_key',
 'dict_key',
 'pos',
 'usage',
 'timestamp',
 'word',
 'stem',
 'lang',
 'category',
 'profileid']

In [14]:
lookup.filter(col.id == col.book_key).glimpse()


Rows: 0
Columns: 7
$ id        <str> 
$ word_key  <str> 
$ book_key  <str> 
$ dict_key  <str> 
$ pos       <str> 
$ usage     <str> 
$ timestamp <i64> 



In [13]:
lookup.filter(col.id != col.book_key).glimpse()

Rows: 2309
Columns: 7
$ id        <str> 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AWUAAAC/CgAA:39898:9', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AWYAAAA/AAAA:40112:10', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AWYAAACNAQAA:40446:8', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AWYAAABYAgAA:40649:12', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AWgAAAAyAAAA:41290:14', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AXUAAAAPAAAA:42717:9', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AXYAAAA+AAAA:42996:10', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AYsAAAACAAAA:46968:8', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AZMAAACdAwAA:50142:13', 'CR!SF1BCTR3G51RB371QMV99NB47P8E:AZgAAABhAQAA:51544:11'
$ word_key  <str> 'fr:règles', 'fr:privées', 'fr:fouet', 'fr:éclairage', 'fr:rébarbative', 'fr:saurai', 'fr:tressés', 'fr:Jadis', 'fr:bousculade', 'fr:bouchées'
$ book_key  <str> 'CR!SF1BCTR3G51RB371QMV99NB47P8E', 'CR!SF1BCTR3G51RB371QMV99NB47P8E', 'CR!SF1BCTR3G51RB371QMV99NB47P8E', 'CR!SF1BCTR3G51RB371QMV99NB47P8E', 'CR!SF1BCTR3G51RB371QMV99NB47P8E', 'CR!SF1BCTR3G51RB371QMV99NB47P8E', 'CR!SF

In [10]:
lookup.head()

id,word_key,book_key,dict_key,pos,usage,timestamp
str,str,str,str,str,str,i64
"""CR!SF1BCTR3G51RB371QMV99NB47P8…","""fr:règles""","""CR!SF1BCTR3G51RB371QMV99NB47P8…","""B00NM4BNTS""","""AWUAAAC/CgAA:39898""","""Les moindres débris de tissu é…",1677623350330
"""CR!SF1BCTR3G51RB371QMV99NB47P8…","""fr:privées""","""CR!SF1BCTR3G51RB371QMV99NB47P8…","""B00NM4BNTS""","""AWYAAAA/AAAA:40112""","""Cette absence presque complète…",1677623397009
"""CR!SF1BCTR3G51RB371QMV99NB47P8…","""fr:fouet""","""CR!SF1BCTR3G51RB371QMV99NB47P8…","""B00NM4BNTS""","""AWYAAACNAQAA:40446""","""Il arriva une ou deux fois qu'…",1677623431007
"""CR!SF1BCTR3G51RB371QMV99NB47P8…","""fr:éclairage""","""CR!SF1BCTR3G51RB371QMV99NB47P8…","""B00NM4BNTS""","""AWYAAABYAgAA:40649""","""Mais, avec la nourriture, l'éc…",1677623467424
"""CR!SF1BCTR3G51RB371QMV99NB47P8…","""fr:rébarbative""","""CR!SF1BCTR3G51RB371QMV99NB47P8…","""B00NM4BNTS""","""AWgAAAAyAAAA:41290""","""Je rejoignis Théa qui avait to…",1677623634330
